# **Ensemble of Local Model-Agnostic Explanation Models for Robust Local Feature Importance Ranking**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from typing import *

import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors

# Explainable AI tools:
import shap
from lime.lime_tabular import LimeTabularExplainer
from alibi.explainers import AnchorTabular # why not used the original anchor package?

from scipy.stats import spearmanr, pearsonr

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam  # Import the Adam optimizer

from tools.topsis import Topsis

2024-11-20 15:40:26.717971: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-20 15:40:26.726143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732128026.735861   22784 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732128026.738935   22784 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-20 15:40:26.749252: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
# Configure pandas output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 40)

# Data Loading and Preprocessing

### Data loading and summarizing

In [4]:
original_data = pd.read_csv('german_credit_data_updated.csv')

# Dataset overview - German Credit Risk (from Kaggle):
# 1. Age (numeric)
# 2. Sex (text: male, female)
# 3. Job (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)
# 4. Housing (text: own, rent, or free)
# 5. Saving accounts (text - little, moderate, quite rich, rich)
# 6. Checking account (numeric, in DM - Deutsch Mark)
# 7. Credit amount (numeric, in DM)
# 8. Duration (numeric, in month)
# 9. Purpose (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)

display(original_data.head())
display(original_data.describe())
display(original_data.info())

# Display the unique values of thprecision=3, e categorical features:
print('Unique values of the categorical features:')
for col in original_data.select_dtypes(include='object'):
    print(f'\t- {col}: {original_data[col].unique()}')

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Credit Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,1
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,2
2,2,49,male,1,own,little,NaN,2096,12,education,1
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,1
4,4,53,male,2,free,little,little,4870,24,car,2


,Unnamed: 0,Age,Job,Credit amount,Duration,Credit Risk
count,954.000000,954.000000,954.000000,954.000000,954.000000,954.000000
mean,476.500000,35.501048,1.909853,3279.112159,20.780922,1.302935
std,275.540378,11.379668,0.649681,2853.315158,12.046483,0.459768
min,0.000000,19.000000,0.000000,250.000000,4.000000,1.000000
25%,238.250000,27.000000,2.000000,1360.250000,12.000000,1.000000
50%,476.500000,33.000000,2.000000,2302.500000,18.000000,1.000000
75%,714.750000,42.000000,2.000000,3975.250000,24.000000,2.000000
max,953.000000,75.000000,3.000000,18424.000000,72.000000,2.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954 entries, 0 to 953
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        954 non-null    int64 
 1   Age               954 non-null    int64 
 2   Sex               954 non-null    object
 3   Job               954 non-null    int64 
 4   Housing           954 non-null    object
 5   Saving accounts   779 non-null    object
 6   Checking account  576 non-null    object
 7   Credit amount     954 non-null    int64 
 8   Duration          954 non-null    int64 
 9   Purpose           954 non-null    object
 10  Credit Risk       954 non-null    int64 
dtypes: int64(6), object(5)
memory usage: 82.1+ KB


None

Unique values of the categorical features:
	- Sex: ['male' 'female']
	- Housing: ['own' 'free' 'rent']
	- Saving accounts: [nan 'little' 'quite rich' 'rich' 'moderate']
	- Checking account: ['little' 'moderate' nan 'rich']
	- Purpose: ['radio/TV' 'education' 'furniture/equipment' 'car' 'business'
 'domestic appliances' 'repairs' 'vacation/others']


### Data preprocessing

In [5]:
preprocessed_data = original_data.copy()

# For savings and checking accounts, we will replace the missing values with 'none':
preprocessed_data['Saving accounts'].fillna('none', inplace=True)
preprocessed_data['Checking account'].fillna('none', inplace=True)

# Dropping index column:
preprocessed_data.drop(columns=['Unnamed: 0'], inplace=True)

# Using pd.dummies to one-hot-encode the categorical features
preprocessed_data["Job"] = preprocessed_data["Job"].map({0: 'unskilled_nonresident', 1: 'unskilled_resident',
                                                         2: 'skilled', 3: 'highlyskilled'})

categorical_features = preprocessed_data.select_dtypes(include='object').columns
numerical_features = preprocessed_data.select_dtypes(include='number').columns.drop('Credit Risk')
print(f'Categorical features: {categorical_features}')
print(f'Numerical features: {numerical_features}')

preprocessed_data = pd.get_dummies(preprocessed_data, columns=categorical_features, dtype='int64')

# Remapping the target variable to 0 and 1:
preprocessed_data['Credit Risk'] = preprocessed_data['Credit Risk'].map({1: 0, 2: 1})

# Make sure all column names are valid python identifiers (important for pd.query() calls):
preprocessed_data.columns = preprocessed_data.columns.str.replace(' ', '_')
preprocessed_data.columns = preprocessed_data.columns.str.replace('/', '_')

# Normalizing the data
scaler = StandardScaler()
scaled_preprocessed_data = scaler.fit_transform(preprocessed_data)

display(preprocessed_data.head())
display(preprocessed_data.info())

display(scaled_preprocessed_data)


Categorical features: Index(['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Purpose'],
      dtype='object')
Numerical features: Index(['Age', 'Credit amount', 'Duration'], dtype='object')


,Age,Credit_amount,Duration,Credit_Risk,Sex_female,Sex_male,Job_highlyskilled,Job_skilled,Job_unskilled_nonresident,Job_unskilled_resident,Housing_free,Housing_own,Housing_rent,Saving_accounts_little,Saving_accounts_moderate,Saving_accounts_none,Saving_accounts_quite_rich,Saving_accounts_rich,Checking_account_little,Checking_account_moderate,Checking_account_none,Checking_account_rich,Purpose_business,Purpose_car,Purpose_domestic_appliances,Purpose_education,Purpose_furniture_equipment,Purpose_radio_TV,Purpose_repairs,Purpose_vacation_others
0,67,1169,6,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,22,5951,48,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,49,2096,12,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,45,7882,42,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,53,4870,24,1,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954 entries, 0 to 953
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Age                          954 non-null    int64
 1   Credit_amount                954 non-null    int64
 2   Duration                     954 non-null    int64
 3   Credit_Risk                  954 non-null    int64
 4   Sex_female                   954 non-null    int64
 5   Sex_male                     954 non-null    int64
 6   Job_highlyskilled            954 non-null    int64
 7   Job_skilled                  954 non-null    int64
 8   Job_unskilled_nonresident    954 non-null    int64
 9   Job_unskilled_resident       954 non-null    int64
 10  Housing_free                 954 non-null    int64
 11  Housing_own                  954 non-null    int64
 12  Housing_rent                 954 non-null    int64
 13  Saving_accounts_little       954 non-null    int64

None

array([[ 2.7694545 , -0.7399179 , -1.22763429, ...,  1.62518349,
        -0.14633276, -0.11286653],
       [-1.18704073,  0.93690642,  2.26068929, ...,  1.62518349,
        -0.14633276, -0.11286653],
       [ 1.18685641, -0.41486224, -0.72930235, ..., -0.61531514,
        -0.14633276, -0.11286653],
       ...,
       [-1.0111965 , -0.39768023,  1.26402541, ..., -0.61531514,
        -0.14633276, -0.11286653],
       [-0.65950803,  0.29240557,  0.26736153, ..., -0.61531514,
        -0.14633276, -0.11286653],
       [-0.83535227,  2.69823821,  1.26402541, ..., -0.61531514,
        -0.14633276, -0.11286653]])

### Splitting the data into training and testing sets:

In [6]:
y = preprocessed_data['Credit_Risk']
X = preprocessed_data.drop(columns='Credit_Risk')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sklearn Model Fitting

In [7]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'ROC AUC: {roc_auc_score(y_test, y_pred)}')

Accuracy: 0.7696335078534031
ROC AUC: 0.6830357142857143


# Algorithm Implementation
Given a dataset, an ML model trained on this dataset (whose predictions we want to explain), and a set of explanation tools from which the aggregate explanation model will be built:

1. Apply each explanation model to the instance whose prediction is to be explained, obtaining a feature importance ranking for each explanation model.
2. Generate a variation of the original dataset by introducing noise.
3. Repeat step 1 on this noisy dataset to obtain a new set of feature importance rankings.
4. Assess the robustness of each explanation model by comparing its feature importance ranking on the original data with the ranking on the noisy data.
5. Finally, compute an aggregate explanation ranking by taking the weighted average of the rankings obtained in step 1, with the weights based on each model's stability (determined in step 4).

### Step 1
Apply each explanation model to the instance whose prediction is to be explained, obtaining a feature importance ranking for each explanation model.

In [8]:
class FeatureImportanceCalculator():
    def __init__(self, clf, X_train: pd.DataFrame | np.ndarray, predict_proba_function: callable = None):
        self.X_train = X_train
        self.clf = clf
        
        if predict_proba_function is not None:
            self.predict_proba_function = predict_proba_function
        elif hasattr(clf, 'predict_proba') and predict_proba_function is None:
            self.predict_proba_function = clf.predict_proba
        else:
            raise ValueError('The classifier does not have a predict_proba method and no predict_proba_function was provided.')
        
        self.anchor_explainer = AnchorTabular(predictor=self.predict_proba_function, feature_names=self.X_train.columns) # TODO: fix parameters
        self.anchor_explainer.fit(self.X_train.values)


    def get_lime_ranking(self, instance_data_row) -> pd.DataFrame:
        """
        Returns a DataFrame with the feature importance ranking using LIME, ordered by abs(importance).
        """
        
        explainer = LimeTabularExplainer(self.X_train.values, feature_names=self.X_train.columns, discretize_continuous=False)
        lime_exp = explainer.explain_instance(instance_data_row, self.predict_proba_function, num_features=len(self.X_train.columns))
        
        ranking = pd.DataFrame(lime_exp.as_list(), columns=['feature', 'score'])

        return ranking
    
    def get_shap_ranking(self, instance_data_row, explainer_type: shap.Explainer = shap.KernelExplainer, **additional_explainer_args) -> pd.DataFrame:
        """
        Returns a DataFrame with the feature importance ranking using SHAP, ordered by abs(importance).
        """
        explainer = explainer_type(self.clf, self.X_train, **additional_explainer_args)
        shap_values = explainer.shap_values(instance_data_row)

        ranking = pd.DataFrame(list(zip(self.X_train.columns, shap_values[:, 0])), columns=['feature', 'score'])
        ranking = ranking.sort_values(by='score', ascending=False, key=lambda x: abs(x)).reset_index(drop=True)
        
        return ranking
    
    def get_anchor_ranking(self, instance_data_row: pd.Series | np.ndarray) -> pd.DataFrame:
        """
        Returns a DataFrame with the feature importance ranking using Anchor, ordered by abs(importance).
        Feature importance is not directly available in the AnchorTabular class. In order to obtain it, we can
        calculate the percentage of rows in the training data that are not covered by the anchor rule. The more
        rows that are not covered, the more important the feature is.
        """

        if isinstance(instance_data_row, pd.Series):
            instance_data_row = instance_data_row.to_numpy()

        feature_importances = {feature: 0 for feature in self.X_train.columns}
        explanation = self.anchor_explainer.explain(instance_data_row)
        
        for rule in explanation.anchor:
            # Extract the feature name from the rule string
            # This method won't work for column names that have spaces in them or that don't contain any letters
            for expression_element in rule.split():
                if any(c.isalpha() for c in expression_element):
                    referenced_feature = expression_element
                    break

            rule_coverage = X_train.query(rule).shape[0] / X_train.shape[0]
            feature_importances[referenced_feature] = 1 - rule_coverage
        
        return pd.DataFrame(list(feature_importances.items()), columns=['feature', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)

In [9]:
# Example usage:
feature_importances = FeatureImportanceCalculator(clf, X_train)
    
sample_idx = 0
lime_ranking = feature_importances.get_lime_ranking(X_test.iloc[sample_idx])
shap_ranking = feature_importances.get_shap_ranking(X_test.iloc[sample_idx], explainer_type=shap.TreeExplainer)
anchor_ranking = feature_importances.get_anchor_ranking(X_test.iloc[sample_idx])

print('LIME ranking:')
display(lime_ranking)
print('SHAP ranking:')
# display(shap_ranking)
print('Anchor ranking:')
display(anchor_ranking)

LIME ranking:


,feature,score
0,Checking_account_none,-0.058835
1,Duration,0.053430
2,Checking_account_little,0.036393
3,Age,-0.024741
4,Checking_account_moderate,0.015632
5,Housing_own,-0.012648
6,Saving_accounts_little,0.010625
7,Sex_male,-0.010128
8,Purpose_radio_TV,-0.008845
9,Credit_amount,0.007636


SHAP ranking:
Anchor ranking:


,feature,score
0,Purpose_furniture_equipment,0.813893
1,Duration,0.804718
2,Age,0.503277
3,Checking_account_little,0.283093
4,Saving_accounts_quite_rich,0.000000
5,Purpose_repairs,0.000000
6,Purpose_radio_TV,0.000000
7,Purpose_education,0.000000
8,Purpose_domestic_appliances,0.000000
9,Purpose_car,0.000000


### Step 2
Generate a variation of the original dataset by introducing noise.

In [10]:
def get_noisy_data_autoencoder(X: pd.DataFrame, categorical_features_names: list[str], encoding_dim: int = 5, num_features_to_replace: int = 2, epochs=500) -> pd.DataFrame:
    """
    Returns a DataFrame containing a noisy variation of the data.

    The noise is generated by swapping the values of a small number of features between a sample and a random close neighbor.
    To determine the neighbors, we use an autoencoder to reduce the dimensionality of the data and then calculate the use the NearestNeightbors algorithm in the reduced space.
    """

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    input_dim = X_scaled.shape[1]

    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)

    autoencoder = Model(inputs=input_layer, outputs=decoded)
    encoder = Model(inputs=input_layer, outputs=encoded)
    autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')
    autoencoder.fit(X_scaled, X_scaled, epochs=epochs, batch_size=32, shuffle=True, validation_split=0.2)
    # Extract hidden layer representation:
    hidden_representation = encoder.predict(X_scaled)

    # Compute Nearest Neighbors using hidden_representation
    nbrs = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(hidden_representation) # TODO: here, hidden_representation is just the autoencoder fit to the scaled X data; see if this is the way to do this
    distances, indices = nbrs.kneighbors(hidden_representation)

    X_noisy = X.copy()

    # Get id's of columns that belong to the same categorical feature (after being one-hot-encodeded);
    # Columns that belong to the same categorical feature start with the same name, and will be treated as a single feature when adding noise.
    categorical_features_indices = [
        [X.columns.get_loc(col_name) for col_name in X.columns if col_name.startswith(feature)]
        for feature in categorical_features_names
    ]

    # Replace features with random neighbor's features
    for i in range(X.shape[0]):  # Iterate over each sample
        available_features_to_replace = list(range(X.shape[1]))
        for j in range(num_features_to_replace):
            # Select features to replace; if the feture selected belong to one of the lists in categorical_features_indices, we will replace all the features in that list
            features_to_replace = np.random.choice(available_features_to_replace, 1)
            for feature_indices in categorical_features_indices:
                if features_to_replace in feature_indices:
                    features_to_replace = feature_indices
                    break
            
            # Remove the selected features from the list of available features to replace
            available_features_to_replace = [f for f in available_features_to_replace if f not in features_to_replace]

            # Choose a random neighbor from the nearest neighbors
            neighbor_idx = np.random.choice(indices[i][1:])

            # Replace the selected features with the neighbor's features
            X_noisy.iloc[i, features_to_replace] = X.iloc[neighbor_idx, features_to_replace]

    return X_noisy

In [11]:
# Example of usage:
display(X.shape)
noisy_data = get_noisy_data_autoencoder(X, categorical_features_names=categorical_features, encoding_dim=5, num_features_to_replace=2, epochs=10)

# Split the noisy data the same way as the original data
# X_train_noisy, X_test_noisy = train_test_split(noisy_data, test_size=0.2, random_state=41) # Split it with the same random_state as the original data
X_train_noisy = noisy_data.loc[X_train.index]
X_test_noisy = noisy_data.loc[X_test.index]
display("Mean Absolute Difference: ", np.mean(np.abs(X - noisy_data)))

(954, 29)

Epoch 1/10


W0000 00:00:1732128028.908517   22784 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2707 - val_loss: 1.2044
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2288 - val_loss: 1.1834
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2396 - val_loss: 1.1636
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2376 - val_loss: 1.1439
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2091 - val_loss: 1.1237
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1681 - val_loss: 1.1032
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1567 - val_loss: 1.0828
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step - loss: 1.1498 - val_loss: 1.0629
Epoch 9/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0952 - val_loss: 1.0437
Epoch 10/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0792 - val_loss: 1.0257
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step


'Mean Absolute Difference: '

5.094954095279404

### Step 3
Repeat step 1 on this noisy dataset to obtain a new set of feature importance rankings.

In [12]:
# Given a single instance, get rankings for original and noisy data, then compare them (next steps).
feature_importances_noisy = FeatureImportanceCalculator(clf, X_train_noisy)

lime_ranking_noisy = feature_importances_noisy.get_lime_ranking(X_test_noisy.iloc[sample_idx])
shap_ranking_noisy = feature_importances_noisy.get_shap_ranking(X_test_noisy.iloc[sample_idx], explainer_type=shap.TreeExplainer)
anchor_ranking_noisy = feature_importances_noisy.get_anchor_ranking(X_test_noisy.iloc[sample_idx])

display(lime_ranking_noisy, shap_ranking_noisy, anchor_ranking_noisy)

,feature,score
0,Checking_account_none,-0.057467
1,Duration,0.057258
2,Checking_account_little,0.035452
3,Age,-0.027223
4,Checking_account_moderate,0.019969
5,Housing_own,-0.013258
6,Purpose_radio_TV,-0.010295
7,Saving_accounts_little,0.008335
8,Housing_free,0.007493
9,Sex_male,-0.006986


,feature,score
0,Checking_account_none,-0.050534
1,Age,0.039863
2,Duration,0.037818
3,Checking_account_little,0.028099
4,Checking_account_moderate,-0.027543
5,Credit_amount,0.019034
6,Purpose_car,-0.013071
7,Saving_accounts_moderate,0.010728
8,Sex_male,0.008038
9,Saving_accounts_little,-0.006773


,feature,score
0,Duration,0.433814
1,Sex_female,0.313237
2,Housing_own,0.296199
3,Checking_account_little,0.283093
4,Age,0.000000
5,Saving_accounts_rich,0.000000
6,Purpose_repairs,0.000000
7,Purpose_radio_TV,0.000000
8,Purpose_furniture_equipment,0.000000
9,Purpose_education,0.000000


### Step 4
Assess the robustness of each explanation model by comparing its feature importance ranking on the original data with the ranking on the noisy data.

In [13]:
def robustness_metrics(ranking_original: pd.DataFrame, ranking_noisy: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a DataFrame with 4 robustness metrics of a given feature importance ranking:
        1. "mean_squared_differece": the mean squared difference between the scores of the original and noisy rankings; (previously called "stability") | Lower is better
        2. "mean_absolute_difference": the mean absolute difference between the scores of the original and noisy rankings; (previously called "sensitivity") | Lower is better
        3. "spearman_correlation": the Spearman correlation | Higher is better
        4. "pearson_correlation": the Pearson correlation | Higher is better
    """
    
    # Align dataframes:
    ranking_original = ranking_original.set_index('feature')
    ranking_noisy = ranking_noisy.set_index('feature')
    ranking_original = ranking_original.reindex(ranking_noisy.index)

    # Compute metrics:
    mean_squared_difference = ((ranking_original['score'] - ranking_noisy['score']) ** 2).mean()
    mean_absolute_difference = np.abs(ranking_original['score'] - ranking_noisy['score']).mean()
    spearman_correlation = spearmanr(ranking_original['score'], ranking_noisy['score']).correlation
    pearson_correlation = pearsonr(ranking_original['score'], ranking_noisy['score'])[0]

    robustness_metrics = pd.DataFrame({
        'mean_squared_difference': [mean_squared_difference],
        'mean_absolute_difference': [mean_absolute_difference],
        'spearman_correlation': [spearman_correlation],
        'pearson_correlation': [pearson_correlation]
    })

    return robustness_metrics

In [14]:
print("Robustness metrics for LIME:")
display(robustness_metrics(lime_ranking, lime_ranking_noisy))

print("Robustness metrics for SHAP:")
display(robustness_metrics(shap_ranking, shap_ranking_noisy))

print("Robustness metrics for Anchor:")
display(robustness_metrics(anchor_ranking, anchor_ranking_noisy))

Robustness metrics for LIME:


,mean_squared_difference,mean_absolute_difference,spearman_correlation,pearson_correlation
0,0.000006,0.002102,0.928571,0.991705


Robustness metrics for SHAP:


,mean_squared_difference,mean_absolute_difference,spearman_correlation,pearson_correlation
0,0.00003,0.003304,0.768568,0.962927


Robustness metrics for Anchor:


,mean_squared_difference,mean_absolute_difference,spearman_correlation,pearson_correlation
0,0.042729,0.079224,0.40137,0.423505


### Step 5

In [15]:
def explain_instance(instance_data_row):
    lime_ranking = feature_importances.get_lime_ranking(instance_data_row)
    lime_ranking_noisy = feature_importances_noisy.get_lime_ranking(instance_data_row)
    lime_instance_metrics = robustness_metrics(lime_ranking, lime_ranking_noisy)

    shap_ranking = feature_importances.get_shap_ranking(instance_data_row, explainer_type=shap.TreeExplainer)
    shap_ranking_noisy = feature_importances_noisy.get_shap_ranking(instance_data_row, explainer_type=shap.TreeExplainer)
    shap_instance_metrics = robustness_metrics(shap_ranking, shap_ranking_noisy)

    anchor_ranking = feature_importances.get_anchor_ranking(instance_data_row)
    anchor_ranking_noisy = feature_importances_noisy.get_anchor_ranking(instance_data_row)
    anchor_instance_metrics = robustness_metrics(anchor_ranking, anchor_ranking_noisy)

    ### Use toposis to assign weights to the explanation models:
    evaluation_matrix = np.array([
        lime_instance_metrics.values.flatten(),
        shap_instance_metrics.values.flatten(),
        anchor_instance_metrics.values.flatten()
    ])

    display(evaluation_matrix)

    robustness_metrics_weights = [
        0.25, # mean squared difference
        0.25, # mean absolute difference
        0.25, # spearman correlation
        0.25  # pearson correlation
    ]

    # if higher value is preferred - True
    # if lower value is preferred - False
    criterias = np.array([
        False,  # For mean_squared_difference, lower is better
        False,  # For mean_absolute_difference, lower is better
        True,   # For spearman_correlation, higher is better
        True    # For pearson_correlation, higher is better
    ])

    t = Topsis(evaluation_matrix, robustness_metrics_weights, criterias, debug=True)
    t.calc()

    display(t.best_similarity)
    # Sujoy suggested using the best_similarity as weights for the explanations

    lime_weight, shap_weight, anchor_weight = t.best_similarity
    result = lime_ranking.merge(shap_ranking, on='feature', how='outer', suffixes=('_lime', '_shap'))
    result = result.merge(anchor_ranking, on='feature', how='outer', suffixes=('_lime', '_shap'))
    result = result.rename(columns={'score': 'score_anchor'})
    result['score_ensemble'] = lime_weight * result['score_lime'] + shap_weight * result['score_shap'] + anchor_weight * result['score_anchor']
    result = result.sort_values(by='score_ensemble', ascending=False).reset_index(drop=True)

    return result


In [16]:
sample_idx = 0
explain_instance(X_test.iloc[sample_idx])

array([[0.00000497, 0.00163388, 0.91182266, 0.99251495],
       [0.00000336, 0.00111344, 0.97746029, 0.99537514],
       [0.02586741, 0.03827903, 0.6890411 , 0.55892702]])

Step 1 - Evaluation Matrix:
 [[0.00000497 0.00163388 0.91182266 0.99251495]
 [0.00000336 0.00111344 0.97746029 0.99537514]
 [0.02586741 0.03827903 0.6890411  0.55892702]]

Step 2 - Normalized Evaluation Matrix:
 [[0.000192   0.04262671 0.60631723 0.65612208]
 [0.00013    0.02904875 0.64996302 0.65801287]
 [0.99999997 0.99866868 0.45817844 0.36949001]]

Step 3 - Weighted Normalized Evaluation Matrix
 [[0.000048   0.01065668 0.15157931 0.16403052]
 [0.0000325  0.00726219 0.16249076 0.16450322]
 [0.24999999 0.24966717 0.11454461 0.0923725 ]]

Step 4 - worst_alternatives | best_alternatives 
 [0.24999999 0.24966717 0.11454461 0.0923725 ]  |  [0.0000325  0.00726219 0.16249076 0.16450322]

Step 5 - Distances to Worst Alternative | Distances to Best Alternative
 [0.35511752 0.35881136 0.        ] [0.01143704 0.         0.35881136]

Step 6 - Similarites to Worst Alternative | Similarities to Best Alternative
 [0.96879853 1.         0.        ] [0.03120147 0.         1.        ]



array([0.03120147, 0.        , 1.        ])

,feature,score_lime,score_shap,score_anchor,score_ensemble
0,Purpose_furniture_equipment,0.000012,0.009065,0.813893,0.813893
1,Age,-0.027337,0.044427,0.503277,0.502424
2,Duration,0.054827,0.051965,0.433814,0.435525
3,Checking_account_little,0.038031,0.030740,0.283093,0.284280
4,Checking_account_moderate,0.016900,-0.025005,0.000000,0.000527
5,Credit_amount,0.008780,0.018809,0.000000,0.000274
6,Saving_accounts_little,0.008481,-0.003987,0.000000,0.000265
7,Purpose_education,0.006638,0.001744,0.000000,0.000207
8,Sex_female,0.005948,0.007021,0.000000,0.000186
9,Purpose_business,0.005631,-0.000601,0.000000,0.000176


### Pevious attempt at step 5 (IGNORE)

### Exploring different XAI Evaluation Metrics

#### Faithfullness

In [17]:
def faithfullness(predict_proba, explain_instance: callable, instance_data_row: pd.Series, len_subset: int = None, iterations: int = 10, baseline_strategy: str = "zeros") -> float:
    dimension = len(instance_data_row)

    importance_sums = []
    delta_fs = []

    f_x = predict_proba(instance_data_row.to_numpy().reshape(1, -1))[0][1]
    display(f_x)
    g_x = explain_instance(instance_data_row)

    for _ in range(iterations):
        # Select a subset of features to perturb
        subset = np.random.choice(instance_data_row.index.values, len_subset if len_subset else dimension/4, replace=False)

        perturbed_instance = instance_data_row.copy()

        if baseline_strategy == "zeros":
            baseline = np.zeros(dimension)  # either mean on all zeros
        elif baseline_strategy == "mean":
            pass # yet to be defined
        perturbed_instance[subset] = baseline[instance_data_row.index.get_indexer(subset)]

        importance_sum = 0
        for feature in subset:
            importance_sum += g_x[g_x['feature'] == feature]['score'].values[0] # should I take the abs value here?
        importance_sums.append(importance_sum)

        f_x_perturbed = predict_proba(perturbed_instance.to_numpy().reshape(1, -1))[0][1]
        delta_f = np.abs(f_x - f_x_perturbed)
        delta_fs.append(delta_f)
    
    # display(importance_sums, delta_fs)
    return pearsonr(importance_sums, delta_fs).statistic
    
        

In [18]:
g = FeatureImportanceCalculator(clf, X_train)
faithfullness(clf.predict_proba, lambda instance: g.get_shap_ranking(instance, explainer_type=shap.TreeExplainer), X_test.iloc[sample_idx], len_subset=10, iterations=1000, baseline_strategy="zeros")

0.18

0.399866252455617

In [19]:
class AutoencoderNoisyDataGenerator():
    def __init__(self, X: pd.DataFrame, ohe_categorical_features_names: list[str], encoding_dim: int = 5, epochs=500):
        self.X = X
        self.categorical_features_names = ohe_categorical_features_names
        self.encoding_dim = encoding_dim
        self.epochs = epochs

        scaler = StandardScaler()
        self.X_scaled = scaler.fit_transform(self.X)
        
        input_dim = self.X_scaled.shape[1]

        input_layer = Input(shape=(input_dim,))
        encoded = Dense(self.encoding_dim, activation='relu')(input_layer)
        decoded = Dense(input_dim, activation='sigmoid')(encoded)

        self.autoencoder = Model(inputs=input_layer, outputs=decoded)
        self.encoder = Model(inputs=input_layer, outputs=encoded)
        self.was_fit = False
        
    
    def fit(self):
        self.autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')
        self.autoencoder.fit(self.X_scaled, self.X_scaled, epochs=self.epochs, batch_size=32, shuffle=True, validation_split=0.2)
        # Extract hidden layer representation:
        self.hidden_representation = self.encoder.predict(self.X_scaled)
        self.was_fit = True


    def generate_noisy_data(self, num_features_to_replace: int = 2) -> pd.DataFrame:
        """
        Returns a DataFrame containing a noisy variation of the data.

        The noise is generated by swapping the values of a small number of features between a sample and a random close neighbor.
        To determine the neighbors, we use an autoencoder to reduce the dimensionality of the data and then calculate the use the NearestNeightbors algorithm in the reduced space.
        """

        if not self.was_fit:
            raise ValueError('The autoencoder has not been fitted yet. Call the fit() method before generating noisy data.')

        # Compute Nearest Neighbors using hidden_representation
        nbrs = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(self.hidden_representation)
        distances, indices = nbrs.kneighbors(self.hidden_representation)

        X_noisy = self.X.copy()

        # Get id's of columns that belong to the same categorical feature (after being one-hot-encodeded);
        # Columns that belong to the same categorical feature start with the same name, and will be treated as a single feature when adding noise.
        categorical_features_indices = [
            [self.X.columns.get_loc(col_name) for col_name in self.X.columns if col_name.startswith(feature)]
            for feature in self.categorical_features_names
        ]

        # Replace features with random neighbor's features
        for i in range(self.X.shape[0]):  # Iterate over each sample
            available_features_to_replace = list(range(self.X.shape[1]))
            for j in range(num_features_to_replace):
                # Select features to replace; if the feture selected belong to one of the lists in categorical_features_indices, we will replace all the features in that list
                features_to_replace = np.random.choice(available_features_to_replace, 1)
                for feature_indices in categorical_features_indices:
                    if features_to_replace in feature_indices:
                        features_to_replace = feature_indices
                        break
                
                # Remove the selected features from the list of available features to replace
                available_features_to_replace = [f for f in available_features_to_replace if f not in features_to_replace]

                # Choose a random neighbor from the nearest neighbors
                neighbor_idx = np.random.choice(indices[i][1:])

                # Replace the selected features with the neighbor's features
                X_noisy.iloc[i, features_to_replace] = self.X.iloc[neighbor_idx, features_to_replace]

        return X_noisy

In [20]:
# Usage Example:
autoencoder_noisy_data_generator = AutoencoderNoisyDataGenerator(X_train, categorical_features, encoding_dim=5, epochs=10)
autoencoder_noisy_data_generator.fit()
noisy_data = autoencoder_noisy_data_generator.generate_noisy_data(num_features_to_replace=2)
display("Mean Absolute Difference: ", np.mean(np.abs(X_train - noisy_data)))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2329 - val_loss: 1.2192
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2264 - val_loss: 1.2028
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2100 - val_loss: 1.1872
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1810 - val_loss: 1.1717
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2186 - val_loss: 1.1556
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1608 - val_loss: 1.1391
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1769 - val_loss: 1.1231
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1139 - val_loss: 1.1075
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0973 - val_loss: 1.0924
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0747 - val_loss: 1.0774
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 664us/step


'Mean Absolute Difference: '

5.946716680977991

In [21]:
class ExplainerWrapper:

    def __init__(self, clf, X_train: pd.DataFrame | np.ndarray, categorical_feature_names: list[str], predict_proba: callable = None):
        self.clf = clf

        if hasattr(clf, 'predict_proba') and predict_proba is None:
            self.predict_proba = clf.predict_proba
        elif predict_proba is not None:
            self.predict_proba = predict_proba
        else:
            raise ValueError('The classifier does not have a predict_proba method and no predict_proba_function was provided.')

        self.X_train = X_train
        self.categorical_feature_names = categorical_feature_names

    
    def explain_instance(self, instance_data_row: pd.Series | np.ndarray) -> pd.DataFrame:
        pass

class LimeWrapper(ExplainerWrapper):

    def __init__(self, clf, X_train: pd.DataFrame | np.ndarray, categorical_feature_names: list[str], predict_proba: callable = None):
        super().__init__(clf, X_train, categorical_feature_names, predict_proba)
        
        self.explainer = LimeTabularExplainer(self.X_train.values, feature_names=self.X_train.columns, discretize_continuous=False)
    
    def explain_instance(self, instance_data_row: pd.Series | np.ndarray) -> pd.DataFrame:
        lime_exp = self.explainer.explain_instance(instance_data_row, self.predict_proba, num_features=len(self.X_train.columns))
        
        ranking = pd.DataFrame(lime_exp.as_list(), columns=['feature', 'score'])
        return ranking

class ShapTabularTreeWrapper(ExplainerWrapper):
    
        def __init__(self, clf, X_train: pd.DataFrame | np.ndarray, categorical_feature_names: list[str], predict_proba: callable = None, **additional_explainer_args):
            super().__init__(clf, X_train, categorical_feature_names, predict_proba)
            
            self.explainer = shap.TreeExplainer(clf, self.X_train, **additional_explainer_args)
        
        def explain_instance(self, instance_data_row: pd.Series | np.ndarray) -> pd.DataFrame:
            shap_values = self.explainer.shap_values(instance_data_row)
    
            ranking = pd.DataFrame(list(zip(self.X_train.columns, shap_values[:, 0])), columns=['feature', 'score'])
            ranking = ranking.sort_values(by='score', ascending=False, key=lambda x: abs(x)).reset_index(drop=True)
            
            return ranking

class AnchorWrapper(ExplainerWrapper):

    def __init__(self, clf, X_train: pd.DataFrame | np.ndarray, categorical_feature_names: list[str], predict_proba: callable = None):
        super().__init__(clf, X_train, categorical_feature_names, predict_proba)
        
        self.explainer = AnchorTabular(predictor=self.predict_proba, feature_names=self.X_train.columns) # TODO: fix parameters
        self.explainer.fit(self.X_train.values)
    
    def explain_instance(self, instance_data_row: pd.Series | np.ndarray) -> pd.DataFrame:
        if isinstance(instance_data_row, pd.Series):
            instance_data_row = instance_data_row.to_numpy()

        feature_importances = {feature: 0 for feature in self.X_train.columns}
        explanation = self.explainer.explain(instance_data_row)
        
        for rule in explanation.anchor:
            # Extract the feature name from the rule string
            # This method won't work for column names that have spaces in them or that don't contain any letters
            for expression_element in rule.split():
                if any(c.isalpha() for c in expression_element):
                    referenced_feature = expression_element
                    break

            rule_coverage = self.X_train.query(rule).shape[0] / self.X_train.shape[0]
            feature_importances[referenced_feature] = 1 - rule_coverage
        
        return pd.DataFrame(list(feature_importances.items()), columns=['feature', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)


In [22]:
class XaiEvaluator:

    def __init__(self, clf, X_train: pd.DataFrame | np.ndarray, ohe_categorical_feature_names: list[str], predict_proba: callable = None, noise_gen_args: dict = {}):
        self.clf = clf
        if hasattr(clf, 'predict_proba') and predict_proba is None:
            self.predict_proba = clf.predict_proba
        elif predict_proba is not None:
            self.predict_proba = predict_proba
        else:
            raise ValueError('The classifier does not have a predict_proba method and no predict_proba_function was provided.')

        self.X_train = X_train
        self.ohe_categorical_feature_names = ohe_categorical_feature_names

        self.categorical_features_indices = [
            [self.X_train.columns.get_loc(col_name) for col_name in self.X_train.columns if col_name.startswith(feature)]
            for feature in self.ohe_categorical_feature_names
        ]

        self.noisy_data_generator = AutoencoderNoisyDataGenerator(X_train, ohe_categorical_feature_names, **noise_gen_args)

        self.was_initialized = False
    
    # Initialization opeations that take a long time to run
    def init(self):
        self.noisy_data_generator.fit()
        self.was_initialized = True
            
    def faithfullness_correlation(self, explainer: ExplainerWrapper | Type[ExplainerWrapper], instance_data_row: pd.Series, len_subset: int = None, iterations: int = 100, baseline_strategy: str = "zeros") -> float:
        if not isinstance(explainer, ExplainerWrapper):
            explainer = explainer(self.clf, self.X_train, self.ohe_categorical_feature_names, predict_proba=self.predict_proba)
        
        dimension = len(instance_data_row)

        importance_sums = []
        delta_fs = []

        f_x = self.predict_proba(instance_data_row.to_numpy().reshape(1, -1))[0][1]
        g_x = explainer.explain_instance(instance_data_row)

        for _ in range(iterations):
            # Select a subset of features to perturb
            subset = np.random.choice(instance_data_row.index.values, len_subset if len_subset else dimension/4, replace=False)

            perturbed_instance = instance_data_row.copy()

            if baseline_strategy == "zeros":
                baseline = np.zeros(dimension)  # either mean on all zeros
            elif baseline_strategy == "mean":
                baseline = np.mean(self.X_train, axis=0)
                for feature_index in self.categorical_features_indices:
                    baseline[feature_index] = 0
                
            perturbed_instance[subset] = baseline[instance_data_row.index.get_indexer(subset)]

            importance_sum = 0
            for feature in subset:
                importance_sum += g_x[g_x['feature'] == feature]['score'].values[0] # should I take the abs value here?
            importance_sums.append(importance_sum)

            f_x_perturbed = self.predict_proba(perturbed_instance.to_numpy().reshape(1, -1))[0][1]
            delta_f = np.abs(f_x - f_x_perturbed)
            delta_fs.append(delta_f)
        
        return pearsonr(importance_sums, delta_fs).statistic
    
    def sensitivity(self, ExplainerType: Type[ExplainerWrapper], instance_data_row: pd.Series, iterations: int = 10):
        if not self.was_initialized:
            raise ValueError('The XaiEvaluator has not been initialized yet. Call the init() method before evaluating sensitivity.')
        
        original_explainer = ExplainerType(self.clf, self.X_train, self.ohe_categorical_feature_names, predict_proba=self.predict_proba)

        mean_squared_differences: list[float] = []
        for _ in range(iterations):
            # Obtain the original explanation:
            original_explanation = original_explainer.explain_instance(instance_data_row)

            # Obtain the noisy explanation:
            noisy_data = self.noisy_data_generator.generate_noisy_data(num_features_to_replace=2)
            noisy_explainer = ExplainerType(self.clf, noisy_data, self.ohe_categorical_feature_names, predict_proba=self.predict_proba)
            noisy_explanation = noisy_explainer.explain_instance(instance_data_row)

            # Compute the mean squared difference between the scores of the original and noisy explanations:
            mean_squared_difference = ((original_explanation['score'] - noisy_explanation['score']) ** 2).mean()
            mean_squared_differences.append(mean_squared_difference)
        
        return np.mean(mean_squared_differences)

    def complexity(self, explainer: ExplainerWrapper | Type[ExplainerWrapper], instance_data_row: pd.Series) -> float:
        if not isinstance(explainer, ExplainerWrapper):
            explainer = explainer(self.clf, self.X_train, self.ohe_categorical_feature_names, predict_proba=self.predict_proba)

        explanation = explainer.explain_instance(instance_data_row)

        def frac_contribution(explanation: pd.DataFrame, i: int) -> float:
            abs_score_sum = explanation['score'].abs().sum()
            return explanation['score'].abs()[i] / abs_score_sum

        sum = 0
        for i in range(explanation.shape[0]):
            fc = frac_contribution(explanation, i)
            sum += fc * np.log(fc) if fc > 0 else 0
            
        return -sum

In [23]:
# Usage example:
g = FeatureImportanceCalculator(clf, X_train)
xai_eval = XaiEvaluator(clf, X_train, categorical_features, noise_gen_args={'encoding_dim': 5, 'epochs': 10})
xai_eval.init()

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2447 - val_loss: 1.2493
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2534 - val_loss: 1.2316
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2402 - val_loss: 1.2142
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2218 - val_loss: 1.1972
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1781 - val_loss: 1.1805
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1531 - val_loss: 1.1641
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1452 - val_loss: 1.1476
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1532 - val_loss: 1.1316
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1181 - val_loss: 1.1161
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1036 - val_loss: 1.1012
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step


In [24]:
faithfullness =  xai_eval.faithfullness_correlation(AnchorWrapper,
                                                    X_test.iloc[sample_idx], len_subset=10, iterations=100, baseline_strategy="mean")
display(faithfullness)
sensitivity = xai_eval.sensitivity(AnchorWrapper, X_test.iloc[sample_idx], iterations=10)
display(sensitivity)

0.4220889429385705

0.0035197446220883487

In [25]:
complexity = xai_eval.complexity(ShapTabularTreeWrapper, X_test.iloc[sample_idx])
display(complexity)

2.516308873975537

### Xai Evaluation Metrics Trends for Selected Explanation Models

In [28]:
shap_sensitivities = []
shap_faithfullnesses = []
shap_complexities = []

lime_sensitivities = []
lime_faithfullnesses = []
lime_complexities = []

anchor_sensitivities = []
anchor_faithfullnesses = []
anchor_complexities = []

shap_exp = ShapTabularTreeWrapper(clf, X_train, categorical_features)
lime_exp = LimeWrapper(clf, X_train, categorical_features)
anchor_exp = AnchorWrapper(clf, X_train, categorical_features)

for i in range(50):
    sample_idx = i
    shap_sensitivities.append(xai_eval.sensitivity(ShapTabularTreeWrapper, X_test.iloc[sample_idx], iterations=10))
    lime_sensitivities.append(xai_eval.sensitivity(LimeWrapper, X_test.iloc[sample_idx], iterations=10))
    anchor_sensitivities.append(xai_eval.sensitivity(AnchorWrapper, X_test.iloc[sample_idx], iterations=10))

    shap_faithfullnesses.append(xai_eval.faithfullness_correlation(shap_exp, X_test.iloc[sample_idx], len_subset=10, iterations=10, baseline_strategy="mean"))
    lime_faithfullnesses.append(xai_eval.faithfullness_correlation(lime_exp, X_test.iloc[sample_idx], len_subset=10, iterations=10, baseline_strategy="mean"))
    anchor_faithfullnesses.append(xai_eval.faithfullness_correlation(anchor_exp, X_test.iloc[sample_idx], len_subset=10, iterations=10, baseline_strategy="mean"))

    shap_complexities.append(xai_eval.complexity(shap_exp, X_test.iloc[sample_idx]))
    lime_complexities.append(xai_eval.complexity(lime_exp, X_test.iloc[sample_idx]))
    anchor_complexities.append(xai_eval.complexity(anchor_exp, X_test.iloc[sample_idx]))

shap_metrics = pd.DataFrame({
    'faithfullness': shap_faithfullnesses,
    'complexity': shap_complexities
})

lime_metrics = pd.DataFrame({
    'faithfullness': lime_faithfullnesses,
    'complexity': lime_complexities
})

anchor_metrics = pd.DataFrame({
    'faithfullness': anchor_faithfullnesses,
    'complexity': anchor_complexities
})

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now ret

In [33]:
display(shap_metrics.describe())
display(lime_metrics.describe())
display(anchor_metrics.describe())

,faithfullness,complexity
count,50.000000,50.000000
mean,0.024205,2.470693
std,0.587730,0.146090
min,-0.827780,2.071901
25%,-0.562122,2.383871
50%,0.032993,2.485882
75%,0.582136,2.559405
max,0.834696,2.731101


,faithfullness,complexity
count,50.000000,50.000000
mean,-0.023739,2.589746
std,0.410476,0.048431
min,-0.801083,2.478361
25%,-0.374738,2.554466
50%,0.045940,2.581921
75%,0.312178,2.625277
max,0.694030,2.709051


,faithfullness,complexity
count,50.000000,50.000000
mean,0.212699,1.373141
std,0.450593,0.555356
min,-0.734714,0.604767
25%,-0.156941,1.049478
50%,0.304438,1.306196
75%,0.628737,1.535797
max,0.939272,2.661137


In [37]:
# Count how many times anchor_metrics['complexity'] is lower than shap_metrics['complexity'] and lime_metrics['complexity']
anchor_complexity_lower_than_shap = anchor_metrics['complexity'] < shap_metrics['complexity']
anchor_complexity_lower_than_lime = anchor_metrics['complexity'] < lime_metrics['complexity']
anchor_lower_than_both = anchor_complexity_lower_than_shap & anchor_complexity_lower_than_lime

display(anchor_lower_than_both.sum())

45